In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# CSV file
csvfile = "resources/autos.csv"

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Text

In [4]:
# Create an engine to a database file called `autos.sqlite`
engine = create_engine("sqlite:///autos.sqlite")

In [5]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [6]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column

Base = declarative_base()

class Autos(Base):
    __tablename__ = 'autos'

    id = Column(Integer, primary_key=True)
    registration_date = Column(Text)
    name = Column(Text)
    plate_number = Column(Text)

    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [7]:
# Use `create_all` to create the demographics table in the database
Base.metadata.create_all(engine)

In [8]:
# Load the csv file into a pandas dataframe
df_of_data_to_insert = pd.read_csv(csvfile,encoding = 'utf8')

In [9]:
# Use Orient='records' to create a list of data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
data = df_of_data_to_insert.to_dict(orient='records')
data[0]

{'name': 'Mark Halsey',
 'plate_number': 'A193958',
 'registration_date': '3/24/16 11:52'}

In [10]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [11]:
# Save the reference to the `autos` table as a variable called `table`
table = sqlalchemy.Table('autos', metadata, autoload=True)

In [12]:
# Use `table.delete()` to remove any existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(table.delete())

In [13]:
# Use `table.insert()` to insert the data into the table
conn.execute(table.insert(), data)

In [14]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from autos limit 5").fetchall()

[(1, '3/24/16 11:52', 'Mark Halsey', 'A193958'),
 (2, '3/24/16 10:58', 'Sharon Hutchinson', 'CTA504'),
 (3, '3/14/16 12:52', 'Nina Parsley', '11N500M'),
 (4, '3/17/16 16:54', 'Hilary Houston', '5EDM948'),
 (5, '3/31/16 17:25', 'Mandy Jackson', '970PZR')]